# <center>PRÁCTICA 2: APRENDIZAJE SUPERVISADO (DengAI)<center>

**Nombre y apellidos:**

**Usuario VIU:**

<font color="green">*\[\[IMPORTANTE: Este notebook debe usarse como plantilla para empezar esta práctica. Antes de entregar el notebook, se deben eliminar todas las líneas de texto entre dobles corchetes como esta.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se debe leer y comprender esta plantilla de principio a fin antes de empezar. En estas líneas con anotaciones se resuelven las dudas y errores más comunes de la elaboración de esta práctica.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todas las secciones definidas en esta plantilla deben estar presentes en la entrega final y respetarse para facilitar la corrección, aunque cada estudiante puede añadir las subsecciones que considere. Los huecos para rellenar al principio del notebook, la tabla de la sección "Conclusiones" y las capturas de pantalla con el nombre de usuario y puntuaciones de DrivenData de la sección "Competición" también son obligatorios.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se debe entregar un único notebook de Jupyter (fichero *.ipynb) con la práctica desarrollada y los resultados ejecutados. Debe estar autocontenido, ya que no se corregirá nada más que este fichero entregado. Las capturas con la puntuación de DrivenData y el nombre de usuario de las pruebas se deben incrustar directamente en la sección "Competición" y no como enlaces externos a ficheros de Internet ni imágenes aparte.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Se debe entregar UN ÚNICO notebook de Jupyter (fichero *.ipynb) con la práctica desarrollada. No se debe entregar ningún otro fichero de imagenes, .csv, .pdf, ni se debe comprimir en un .zip. El fichero .ipynb debe estar autocontenido, ya que no se corregirá nada más que este fichero entregado.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todo el notebook debe ser ejecutable. En el notebook se deben dejar los resultados de los experimentos, reproducibles por el profesor en su sesión de Google Colab. Se aconseja encarecidamente que se fije una semilla aleatoria única para todos los procesos estocásticos que se vayan a llevar a cabo.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: No se debe ocultar ninguna celda ni ningún grupo de celdas. Todas las secciones del notebook, con todos los resultados, deben estar completamente desplegadas, independientemente del tamaño que ocupen.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: No se debe utilizar el color blanco para texto, ya que el profesor corregirá la práctica con fondo blanco y es posible que haya texto que pase desapercibido con su configuración de colores.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: El fichero *.ipynb, antes de corregirse, se pasará por un detector de plagios específico para estos notebooks de Jupyter con el fin de evitar cualquier tipo de fraude. Los resultados de este detector se revisarán manualmente por lo que se sugiere que el alumno especifique la fuente de los algoritmos (si son de la web) a fin de evitar problemas (no hará falta si son pequeños fragmentos de otras prácticas en las que participó el mismo alumno). Si se demuestra copia de toda o parte de esta práctica de otros alumnos de esta asignatura, en la convocatoria/edición de la asignatura actual o cualquiera anterior de la asignatura, implicará un suspenso con 0 para todas las partes implicadas y la apertura de un expediente.\]\]*</font>

---
# Resumen
---

<font color="green">*\[\[Describir qué se va a hacer en esta práctica, mencionar las diferentes fases en las que se ha dividido la experimentación, así como las técnicas que se van a explicar en las siguientes secciones. Resumir también, muy brevemente, los principales resultados obtenidos.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Será necesario registrarse en la competición de DengAI de DrivenData para poder descargar el dataset: https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/ \]\]*</font>

---
# Inicialización
---

In [ ]:
# Imports generales
import pandas as pd
import io
from google.colab import files
import numpy as np
import time

seed = 42  # Semilla aleatoria arbitraria y constante a incluir en los algoritmos estocásticos para que los experimentos sean siempre reproducibles por el profesor.
# OJO: En los experimentos estocásticos que requieran varias iteraciones con distintas semillas, podéis incorporarla como seed+1, seed+2, etc.

def upload_files (index_fields=None):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)

    return df

def download_predictions(test, pred_total_cases):
  file_name = "practica2_pred_" + time.strftime("%Y") + time.strftime("%m") + time.strftime("%d") + time.strftime("%H") + time.strftime("%M") + time.strftime("%S") + ".csv"
  predictions = pd.DataFrame({ 'city': test['city'], 'year': test['year'], 'weekofyear': test['weekofyear'], 'total_cases': pred_total_cases})
  with open(file_name, 'w') as f:
    predictions.to_csv(f, index = False)
  files.download(file_name)
  print("Downloaded " + file_name)

In [ ]:
# Subir el conjunto de entrenamiento sin variable objetivo (dengue_features_train.csv)
train_feat = upload_files()
print(train_feat.shape)
train_feat.head()

In [ ]:
# Subir la variable objetivo total_cases del conjunto de entrenamiento (dengue_labels_train.csv)
train_labels = upload_files()

# Unificar el conjunto de entrenamiento (características + variable objetivo)
train = pd.merge(train_feat,train_labels,on=['city', 'year', 'weekofyear'])
print(train.shape)
train.head()

In [ ]:
# Subir el conjunto de test sin variable objetivo (dengue_features_test.csv)
test = upload_files()
print(test.shape)
test.head()

---
# Reexploración
---

<font color="green">*\[\[Se hará una expansión del análisis exploratorio razonado del dataset que se hizo en la Práctica 1, ahora teniendo en cuenta también la variable objetivo. Las conclusiones de esta expansión del análisis podrían servir para refinar la selección de características, generar características derivadas adicionales, descartar outliers, así como establecer un orden y estrategia de experimentación que después se respetará en la sección "Predicción".\]\]*</font>

<font color="green">*\[\[IMPORTANTE: DrivenData ofrece 2 conjuntos de datos de DengAI: training y test. Training tiene la variable objetivo "total_cases", que es la que los modelos intentarán predecir. Test, sin embargo, no tiene esas etiquetas y no estarán nunca disponibles. La forma de calcular el error sobre el conjunto de test es mediante el envío de un fichero .csv con las predicciones del conjunto de Test a la web de DrivenData. Solo esta web puede calcular el error sobre el conjunto de test. Los estudiantes solo podrán calcular, en sus ordenadores, el error sobre el conjunto de validación (que se obtendría simplemente mediante las folds del cross-validation, o bien separando manualmente una parte de los datos de training para este fin).\]\]*</font>

<font color="green">*\[\[IMPORTANTE: No basta con ejecutar los algoritmos y mostrar el resultado. Hay que comparar lo obtenido con los experimentos que se han hecho e ir extrayendo deducciones acerca de por qué un curso de experimentación fue o será mejor que otro.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Todos los gráficos que se representen en este notebook deben indicar claramente el título de cada eje de coordenadas.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: En este enlace se puede ver una descripción de qué significan las características del dataset: https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/page/82/ \]\]*</font>

<font color="green">*\[\[Aquí irían todas las funciones y transformaciones que permitirán el uso de los algoritmos de ML sobre el dataset.\]\]*</font>

In [ ]:
# EJEMPLO: Eliminar variable week_start_date por ser de tipo cadena de texto sin formato
train.drop("week_start_date", axis = 1, inplace = True)
test.drop("week_start_date", axis = 1, inplace = True)

<font color="green">*\[\[Rellenar valores perdidos.\]\]*</font>

In [ ]:
# EJEMPLO: Contar valores perdidos
pd.isnull(train).sum()

In [ ]:
# EJEMPLO: Completar valores perdidos con método ffill
# Alternativa: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
train.fillna(method='ffill', inplace=True)
pd.isnull(train).any()

In [ ]:
# EJEMPLO: Completar valores perdidos con método ffill
# Alternativa: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
test.fillna(method='ffill', inplace=True)
pd.isnull(test).any()

<font color="green">*\[\[Transformar variables categóricas si hace falta.\]\]*</font>

In [ ]:
# EJEMPLO: Binarizador para la variable categórica city en el conjunto de entrenamiento y test
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
train['city_bin'] = lb.fit_transform(train['city'])
test['city_bin'] = lb.fit_transform(test['city'])

train.tail()

In [ ]:
# EJEMPLO: Conjunto -aleatorio- de características solo para ilustrar los siguientes pasos
# IMPORTANTE: no tomar esta selección aleatoria como referencia
selected_features = ['reanalysis_specific_humidity_g_per_kg', 'reanalysis_dew_point_temp_k', 'station_avg_temp_c', 'station_min_temp_c']

<font color="green">*\[\[Puede ser conveniente normalizar los datos. Primero se normalizaría el conjunto de entrenamiento. Después el de test, en base a la normalización hecha para el de entrenamiento.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: La necesidad de normalizar depende del problema que se esté tratando y de los datos. Hay casos en los que podría llegar a ser perjudicial o no efectiva.\]\]*</font>

In [ ]:
# EJEMPLO: Normalización con MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train[selected_features])
X_test = scaler.transform(test[selected_features])
y_train = train['total_cases']
X_test

---
# Predicción
---

<font color="green">*\[\[Esta es la sección más importante de esta práctica. Aquí se irán describiendo y ejecutando los distintos experimentos, incluyendo la optimización de hiperparámetros, intercalando celdas de texto y código. Recordad que pueden ser tipos de algoritmos como los que hemos visto en clase, conjuntos de modelos como los que hemos estudiado de árboles, conjuntos de modelos ad-hoc con pre y/o postprocesado, u otros tipos de algoritmos no vistos en clase pero distintos de la familia de la regresión lineal, regresión logística, etc.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Cada modelo descrito debe tener un nombre (RandomForest-A, RandomForestB... por ejemplo) que sirva para identificarlo en la tabla resumen de la sección "Conclusiones".\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Al igual que en el resto de análisis, no sirve ejecutar los algoritmos sin más, como si fueran una caja negra, sino que hay que discutir y razonar los resultados de forma que vayan guiando el discurso de la práctica en cada paso siguiente.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Asimismo, cada prueba descrita debe figurar en el notebook y ser ejecutable. No sirve comentar o mostrar resultados de pruebas que no estén programadas en este mismo notebook y que sean reproducibles por el profesor.\]\]*</font>

<font color="green">*\[\[Será necesario probar y describir en la memoria varios algoritmos de ML, analizar los resultados, replantear las características seleccionadas (si procede), elegir al menos un algoritmo prometedor de entre los probados y por último optimizar bien sus hiperparámetros.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: El algoritmo LinearRegression aquí mostrado como ejemplo NO se tendrá en cuenta a la hora de corregir. Únicamente podría ser útil para tener una línea de base sobre la que comparar los algoritmos que hemos visto en clase u otros de ML que el estudiante quiera explorar aquí.\]\]*</font>

In [ ]:
# EJEMPLO: Ejecución del algoritmo LinearRegression y generación de su modelo
# NO SE TENDRÁ EN CUENTA PARA EVALUAR LA PRÁCTICA 2, TAMPOCO OTROS LOCO REGRESIÓN LOGÍSTICA NI DE LA FAMILIA DENTRO DE import linear_model
# Nombre del modelo: "LinearRegressionA"
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

# Predicción sobre el conjunto de test
y_pred = model.predict(X_test)
y_pred

<font color="green">*\[\[Postprocesar el resultado.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: DrivenData necesita que el orden de las predicciones sea el mismo que en el fichero de test, así como que se una predicción por cada instancia de test. Sino, la web no aceptará el fichero de la submission.\]\]*</font>

In [ ]:
# EJEMPLO: Redondear el resultado y convertirlo a entero
y = np.rint(y_pred) # round
y = y.astype(int) # cast to int
res = np.hstack(y)

# EJEMPLO: Cambiar valores negativos a cero
res[y_pred < 0] = 0
res

<font color="green">*\[\[Para evaluar el MAE en el conjunto de test (o lo que es lo mismo, la puntuación de la competición), será necesario registrarse en la competición e ir subiendo a DrivenData ( https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/ ) las predicciones de los modelos en un fichero .csv que respete el Submission Format (máximo 3 veces al día). La puntuación devuelta por DrivenData corresponde precisamente con el MAE sobre el conjunto de test.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Hacer submissions a DrivenData con las predicciones es imprescindible en este caso, ya que el MAE sobre el conjunto de test no lo puede calcular el estudiante, debido a que el conjunto de test no tiene las etiquetas (variable objetivo). Estas etiquetas existen, pero son secretas y solo las conocen los organizadores de la competición de DrivenData. Sino, los participantes podrían enviar simplemente un .csv con las etiquetas y obtener una puntuación perfecta sin hacer nada de ML.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: DrivenData necesita que el orden de las predicciones sea el mismo que en el fichero de test, así como que se una predicción por cada instancia de test. Sino, la web no aceptará el fichero de la submission.\]\]*</font>

In [ ]:
download_predictions(test, res)

---
# Conclusiones
---

<font color="green">*\[\[Plantilla con ejemplos de la tabla que hay que rellenar resumiendo todos los experimentos descritos en la memoria. Los identificadores de cada modelo deben coincidir con los desarrollados en la sección "Predicción". Un experimento también se consideraría un conjunto de modelos ad hoc con pre y/o postprocesado.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Los resultados de MAE deben tener 4 decimales.\]\]*</font>

| Modelo | MAE test (DrivenData) | MAE validación | Algoritmo | Hiperparámetros | Descripción
| --- | --- | --- | --- | --- | --- |
| Final | 26,1234 | 20,1234 | Algoritmo1 | hp1=0,2; hp2=3 | ....
| A | 27,1234 | 15,1234 | Algoritmo1 | hp1=0,2; hp2=3 | ...
| B | 28,1234 | 18,1234 | Algoritmo2 | hp1=0,2; hp2=3 | ...
| ...


**Modelo final elegido**
* Nombre/Identificador:
* MAE test (DrivenData):
* MAE validación:
* Algoritmo ML:
* Hiperparámetros:
* Lista de características:
* Resumen del dataset (transformaciones aplicadas, etc.):

<font color="green">*\[\[Discusión final de los resultados de todos los modelos descritos en la memoria, justificando la elección del modelo final.\]\]*</font>

<font color="green">*\[\[IMPORTANTE: Para esta práctica, una puntuación del modelo final mayor o igual a 28,9111 (ya que es la que se obtiene simplemente ejecutando esta plantilla) previsiblemente implicará un suspenso. Una puntuación final menor que 25 se considera muy buena. Revisar la rúbrica de esta práctica para entender la puntuación final esperada durante la corrección.\]\]*</font>

<font color="green">*\[\[Discutir trabajo futuro: Mejoras del trabajo realizado que podrían llevarse a cabo pero que no se consideran parte de esta entrega.\]\]*</font>

---
# Competición
---

<font color="green">*\[\[Incrustar aquí una o varias capturas de pantalla de DrivenData con un listado de puntuaciones obtenidas en el que aparezca (resaltada) la correspondiente al modelo final, así como los nombres de usuario
 utilizados en DrivenData (pueden ser varios). OJO, las imágenes deben estar incrustadas directamente en el cuaderno de Jupyter, no ser enlaces a ficheros externos de Drive u otra cosa similar, ni por supuesto entregarlas como ficheros de imágenes a parte, ya que solamente se puede entregar este fichero .ipynb.\]\]*</font>

 <font color="green">*\[\[IMPORTANTE: No incrustar la captura de pantalla con el nombre de usuario y el resultado del mejor modelo implicará no puntuar en el apartado destinado al MAE.\]\]*</font>